Drug Side-Effect Mapping from SIDER

In [1]:
import pandas as pd

In [ ]:
sider_atc = pd.read_csv(r"...path_to\RDsqr-KG\Preprocessed_datasets\drug_atc_sider.csv")
drugbank_atc = pd.read_csv(r"...path_to\RDsqr-KG\Preprocessed_datasets\drug_atc_drugbank.csv")
drug_se = pd.read_csv(r"...path_to\RDsqr-KG\Preprocessed_datasets\drug_se.csv")
drug_status = pd.read_csv(r"...path_to\RDsqr-KG\Preprocessed_datasets\drug_status.csv")
drug_map = pd.read_csv(r"...path_to\RDsqr-KG\Datasets\Vocabulary_links\drug links.csv")


In [3]:
sider_atc['CID'] = sider_atc['CID'].astype(str)
drug_se['CID'] = drug_se['CID'].astype(str)

In [4]:
df_atcdb = pd.merge(sider_atc, drugbank_atc, left_on='atc', right_on='atc_code', how='left')
df_atcdb = df_atcdb.rename(columns={'drugbank_id': 'head_id'})

In [5]:
drug_status.head(1)

,drug_name,include,inclusion_types
0,DB00002,True,approved


In [6]:
df_merged = pd.merge(df_atcdb, drug_status, left_on='head_id', right_on='drug_name', how='left')

In [7]:
df_merged = pd.merge(df_merged, drug_map, left_on='head_id', right_on='DrugBank ID', how='left')
df_merged = df_merged.rename(columns={'Name': 'head_name'})

In [8]:
df_merged.head(1)

,CID,atc,Unnamed: 0,head_id,atc_code,drug_name,include,inclusion_types,DrugBank ID,head_name,...,GenBank ID,DPD ID,RxList Link,Pdrhealth Link,Wikipedia ID,Drugs.com Link,NDC ID,ChemSpider ID,BindingDB ID,TTD ID
0,100000119,L03AA03,3014.0,DB12525,L03AA03,DB12525,True,investigational,DB12525,Molgramostim,...,NaN,NaN,NaN,NaN,Molgramostim,NaN,NaN,NaN,NaN,NaN


In [9]:
df_merged =df_merged[['CID','atc','head_id','atc_code','drug_name','head_name']]

In [10]:
df_drug_se = pd.merge(df_merged, drug_se, left_on='CID', right_on='CID', how='left')
df_drug_se = df_drug_se.rename(columns={'UMLS_meddra': 'tail_id', 'side_effect':'tail_name'})

In [11]:
df_drug_se['head_type'] = 'drug'
df_drug_se['head_ref'] = 'DrugBank'
df_drug_se['tail_type'] = 'side_effect'
df_drug_se['tail_ref'] = 'UMLS'
df_drug_se['predicate'] = 'dse'

In [12]:
df_drug_se = df_drug_se[['head_id', 'head_name', 'head_type','head_ref','tail_id', 'tail_name','tail_type','tail_ref','predicate']]

In [13]:
df_drug_se.shape

(384862, 9)

In [14]:
df_drug_se.isna().sum()

head_id      119069
head_name    119069
head_type         0
head_ref          0
tail_id        1069
tail_name         0
tail_type         0
tail_ref          0
predicate         0
dtype: int64

In [15]:
df_drug_se.dropna(inplace=True)

In [16]:
df_drug_se.isna().sum()

head_id      0
head_name    0
head_type    0
head_ref     0
tail_id      0
tail_name    0
tail_type    0
tail_ref     0
predicate    0
dtype: int64

In [17]:
df_drug_se.drop_duplicates(inplace=True)

In [18]:
df_drug_se.shape

(113799, 9)

In [19]:
df_drug_se.head(1)

,head_id,head_name,head_type,head_ref,tail_id,tail_name,tail_type,tail_ref,predicate
0,DB12525,Molgramostim,drug,DrugBank,C0002792,Anaphylactic shock,side_effect,UMLS,dse


In [20]:
#df_drug_se.dtypes

In [ ]:
df_drug_se.to_csv(r"...path_to\RDsqr-KG\KG_datasets\dse.csv", index=False)